### Deep Structured Semantic Model - Triplet Hinge Loss
This notebook is used to train the Deep Structured Semantic Model (DSSM) using the Yelp dataset. The DSSM model is used to predict user and business interest based on the user and business features. The model is trained using the Triplet Hinge Loss as the loss function according the the paper from Facebook. The goal is to separate the positive and negative samples by a margin, so that the positive samples are closer to the anchor than the negative samples. 

#### Pre-requisites
1. Have the processed Yelp dataset in the `../../data/processed_data/yelp_data` folder.
2. Have the virtual environment setup and used for the notebook.

#### Output
1. `user_model.keras` - The trained user model for retrieval. 
2. `user_id_encoder.pkl` - The label encoder for the user id.
3. `user_scaler.pkl` - The scaler for the user features.
4. `business_model.keras` - The trained business model for retrieval.
5. `business_id_encoder.pkl` - The label encoder for the business id.
6. `categories_encoder.pkl` - The label encoder for the business categories.
7. `business_scaler.pkl` - The scaler for the business features.
    
#### Move to Production
1. Before moving to production, the model should be indexed in the `DSSM Index (Faiss).ipynb` notebook.

In [39]:
from general_program import *

In [40]:
categories_encoder = LabelEncoder()
categories_encoder.fit(list(unique_categories))
user_id_encoder = LabelEncoder()
business_id_encoder = LabelEncoder()

user_scaler = StandardScaler()
business_scaler = StandardScaler()

In [41]:
user_df, business_df, review_df, user_continuous_features_scaled, business_continuous_features_scaled, num_users, num_businesses, num_categories = prepare_data(user_df, business_df, review_df, categories_df, user_id_encoder, business_id_encoder, categories_encoder, user_scaler, business_scaler)

c:\Users\yueny\.virtualenvs\content-recommendation-0SgTkEMC\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\yueny\.virtualenvs\content-recommendation-0SgTkEMC\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
# # Example: Encode 'city' as a discrete feature for businesses
# business_city_encoder = LabelEncoder()
# business_df['city_encoded'] = business_city_encoder.fit_transform(business_df['city'])

# # Save number of unique cities for embedding input_dim
# num_cities = business_df['city_encoded'].max() + 1

In [43]:
def create_embedding_layer(input_dim, output_dim, name):
    """Reusable function to create an embedding layer."""
    return layers.Embedding(input_dim=input_dim, output_dim=output_dim, name=f"{name}_embedding")

# Create embedding layers
user_id_embedding = create_embedding_layer(num_users, 16, "user_id")
business_id_embedding = create_embedding_layer(num_businesses, 16, "business_id")
category_embedding = create_embedding_layer(num_categories, 16, "category")

In [44]:
# Example aggregation function for TensorFlow (Mean Pooling)
def aggregate_category_embeddings(category_indices):
    category_indices = tf.constant(category_indices, dtype=tf.int32)
    embeddings = category_embedding(category_indices)
    return tf.reduce_mean(embeddings, axis=0)

In [45]:
def user_tower(continuous_dim):
    # Inputs
    user_id_input = layers.Input(shape=(1,), name="user_id")
    user_continuous_input = layers.Input(shape=(continuous_dim,), name="user_continuous")

    # Embedding
    user_id_embedded = user_id_embedding(user_id_input)
    user_id_embedded = layers.Flatten()(user_id_embedded)

    # Combine
    concat = layers.Concatenate()([user_id_embedded, user_continuous_input])
    x = layers.Dense(64, activation='relu')(concat)
    x = layers.Dense(32, activation='relu')(x)
    user_embedding = layers.Dense(16, activation=None, name="user_embedding")(x)

    return Model([user_id_input, user_continuous_input], user_embedding, name="UserTower")


In [46]:
def item_tower(continuous_dim):
    # Inputs
    business_id_input = layers.Input(shape=(1,), name="business_id")
    business_continuous_input = layers.Input(shape=(continuous_dim,), name="business_continuous")

    # Embedding
    business_id_embedded = business_id_embedding(business_id_input)
    business_id_embedded = layers.Flatten()(business_id_embedded)

    category_input = layers.Input(shape=(None,), dtype="int32", name="category_indices")  # Variable-length input
    category_embeddings = category_embedding(category_input)
    # aggregated_category_embedding = layers.Lambda(lambda x: tf.reduce_mean(x, axis=1), name="category_pooling", output_shape=(16,))(category_embeddings)
    aggregated_category_embedding = CategoryPoolingLayer(name="category_pooling")(category_embeddings)

    # Combine
    concat = layers.Concatenate()([business_id_embedded, aggregated_category_embedding, business_continuous_input])

    x = layers.Dense(64, activation='relu')(concat)
    x = layers.Dense(32, activation='relu')(x)
    business_embedding = layers.Dense(16, activation=None, name="business_embedding")(x)

    return Model([business_id_input, category_input, business_continuous_input], business_embedding, name="ItemTower")


In [47]:
# Triplet loss function
def triplet_hinge_loss(margin=1.0):
    def loss(y_true, y_pred):
        # y_pred shape: (batch_size, 3, embedding_dim)
        anchor, positive, negative = tf.unstack(y_pred, num=3, axis=1)
        
        # Compute pairwise distances
        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        
        # Hinge loss: max(0, pos_dist - neg_dist + margin)
        return tf.reduce_mean(tf.maximum(pos_dist - neg_dist + margin, 0.0))
    return loss

In [48]:
# Function to generate triplets
def generate_triplets(review_df, num_neg_samples=1):
    triplets = []
    grouped = review_df.groupby('user_id_encoded')

    for user_id, group in grouped:
        positive_samples = group[group['label'] == 1]
        negative_samples = group[group['label'] == 0]
        
        if positive_samples.empty or negative_samples.empty:
            continue  # Skip users without both positive and negative samples
        
        for _, pos_row in positive_samples.iterrows():
            for _, neg_row in negative_samples.sample(num_neg_samples, replace=True).iterrows():
                triplets.append((
                    user_id,
                    pos_row['business_id_encoded'],
                    neg_row['business_id_encoded']
                ))
    
    return np.array(triplets)

In [49]:
# Split review_df into train and test sets
train_df, test_df = train_test_split(review_df, test_size=0.2, random_state=42)

# Generate triplets for training and testing
train_triplets = generate_triplets(train_df)

In [50]:
# Prepare train and test inputs
def prepare_triplet_inputs(triplets, user_features, business_features, business_category_map, max_category_length=MAX_CATEGORY_LENGTH):
    # Replace NaN values with empty lists in `business_category_map`
    business_category_map = business_category_map.apply(lambda x: x if isinstance(x, list) else [])

    anchor_indices = triplets[:, 0]
    positive_indices = triplets[:, 1]
    negative_indices = triplets[:, 2]

    anchor_features = [anchor_indices, user_features.take(anchor_indices, axis=0).values]
    positive_features = [
        positive_indices,
        pad_sequences(business_category_map.loc[positive_indices].tolist(), maxlen=max_category_length, padding="post"),
        business_features.take(positive_indices, axis=0).values, 

]
    negative_features = [
        negative_indices, 
        pad_sequences(business_category_map.loc[negative_indices].tolist(), maxlen=max_category_length, padding="post"),
        business_features.take(negative_indices, axis=0).values,
    ]

    return [
        anchor_features[0], anchor_features[1],
        positive_features[0], positive_features[1], positive_features[2],
        negative_features[0], negative_features[1], negative_features[2]
    ]

business_category_map = business_df.set_index('business_id_encoded')['category_encoded']

max_category_length = MAX_CATEGORY_LENGTH

train_inputs = prepare_triplet_inputs(train_triplets, user_continuous_features_scaled, business_continuous_features_scaled, business_category_map, max_category_length)

In [51]:
# Instantiate towers
user_model = user_tower(user_continuous_features_scaled.shape[1])
item_model = item_tower(business_continuous_features_scaled.shape[1])

# Define inputs for user and business towers
user_inputs_model = [Input(shape=(1,), dtype=tf.int32, name="user_id_input"),
                     Input(shape=(user_continuous_features_scaled.shape[1],), name="user_cont_features_input")]

positive_inputs_model = [
    Input(shape=(1,), dtype=tf.int32, name="positive_id_input"),
    Input(shape=(max_category_length,), dtype=tf.int32, name="positive_category_input"),
    Input(shape=(business_continuous_features_scaled.shape[1],), name="positive_cont_features_input")
]

negative_inputs_model = [
    Input(shape=(1,), dtype=tf.int32, name="negative_id_input"),
    Input(shape=(max_category_length,), dtype=tf.int32, name="negative_category_input"),
    Input(shape=(business_continuous_features_scaled.shape[1],), name="negative_cont_features_input")
]

# Generate embeddings
anchor_embedding = user_model(user_inputs_model)
positive_embedding = item_model(positive_inputs_model)
negative_embedding = item_model(negative_inputs_model)

In [52]:
def stack_embeddings(embeddings):
    # Unpack the embeddings from the list
    anchor, positive, negative = embeddings
    return tf.stack([anchor, positive, negative], axis=1)

triplet_embeddings = Lambda(stack_embeddings, name="triplet_embeddings")(
    [anchor_embedding, positive_embedding, negative_embedding]
)

In [ ]:
all_inputs = user_inputs_model + positive_inputs_model + negative_inputs_model
# Build the model
triplet_model = Model(
    inputs= all_inputs,
    outputs=triplet_embeddings,
    name="triplet_model"
)

# Compile with triplet loss
triplet_model.compile(
    optimizer='adam',
    loss=triplet_hinge_loss(margin=0.2)
)

In [ ]:
# Train the triplet model
triplet_model.fit(
    x=train_inputs,
    y=np.zeros(len(train_inputs[0])),  # Dummy labels, as loss is computed from embeddings
    batch_size=32,
    # epochs=10,
    epochs=3,
    verbose=1
)


Epoch 1/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 193s 15ms/step - loss: 1.1598
Epoch 2/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 185s 14ms/step - loss: 0.5441
Epoch 3/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 185s 14ms/step - loss: 0.3176
Epoch 4/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 188s 15ms/step - loss: 0.2114
Epoch 5/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 187s 15ms/step - loss: 0.1520
Epoch 6/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 188s 15ms/step - loss: 0.1175
Epoch 7/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 189s 15ms/step - loss: 0.0925
Epoch 8/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 189s 15ms/step - loss: 0.0761
Epoch 9/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 190s 15ms/step - loss: 0.0636
Epoch 10/10
12868/12868 ━━━━━━━━━━━━━━━━━━━━ 190s 15ms/step - loss: 0.0529


In [ ]:
save_folder_path = 'Saved_Triplet_Hinge_Loss/'

# Save the models
user_model.save(save_folder_path + 'user_model.keras')
item_model.save(save_folder_path + 'item_model.keras')

# Save the label encoders
with open(save_folder_path + 'user_id_encoder.pkl', 'wb') as f:
    pickle.dump(user_id_encoder, f)

with open(save_folder_path + 'business_id_encoder.pkl', 'wb') as f:
    pickle.dump(business_id_encoder, f)

with open(save_folder_path + 'categories_encoder.pkl', 'wb') as f:
    pickle.dump(categories_encoder, f)
    
# Save the scalers
with open(save_folder_path + 'user_scaler.pkl', 'wb') as f:
    pickle.dump(user_scaler, f)

with open(save_folder_path + 'business_scaler.pkl', 'wb') as f:
    pickle.dump(business_scaler, f)